# LSDL CUB, Homework 1. Robust fine-tuning of CLIP [10 pts]

Your main goal in this home assignment is to implement the [Lipsum-FT](https://openreview.net/attachment?id=2JF8mJRJ7M&name=pdf) method for robust fine-tuning of CLIP.

Rules for the assignment:

- We will be using the same dataset, [DomainNet](https://ai.bu.edu/M3SDA/) (also can be found [here](https://huggingface.co/datasets/wltjr1007/DomainNet)), as in the original paper. Use the **Real** domain as in-distribution (ID) data and the rest of domains as out-of-distribution (OOD) data. Use the **Real** train split for training and test splits for evaluation on all of the domains.

- If training takes too much time, you may select a subset (i.e., 50% or 33%) of training data instead of the full split.

- `ViT-B/16` backbone is recommended.

- In order to **pass the assignment**, you need to plot a Pareto front (i.e., ID-OOD plot like Figure 1 from this [paper](https://arxiv.org/pdf/2109.01903)). We will be using 5 OOD domains, so you need to plot 5 Pareto fronts, one for each distribution shift.

- You may use any code from the [seminar](https://github.com/isadrtdinov/lsdl-cub/tree/2025/week01-finetune/seminar). Also, you may code your training pipelines either in pure PyTorch or combinine it with [huggingface](https://huggingface.co/) libraries. Additionally, you may find [`clip`](https://github.com/openai/CLIP) and [`wise-ft`](https://github.com/mlfoundations/wise-ft) repos useful.

- Do not use the implemention from the authors or any publicly available implementations of this method.

- It will be much easier to check your assigment if you maintain a clear code structure (e.g., put different blocks of code into separate files, add necessary coments, etc).

## 1. Zero-shot model [1 pts]

Create a zero-shot model on top of pre-trained CLIP and evaluate it on **Real** test set (ID accuracy) and on 5 distribution shifts: **Clipart**, **Infograph**, **Painting**, **Quickdraw**, **Sketch** (OOD accuracy).

In [ ]:
# Install CLIP if not already installed
import subprocess
import sys

try:
    import clip
except ImportError:
    print("Installing CLIP...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "git+https://github.com/openai/CLIP.git"])
    import clip

import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from datasets import load_dataset
import numpy as np
from tqdm import tqdm

# Load pre-trained CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model, preprocess = clip.load("ViT-B/16", device=device)

# Load DomainNet dataset
print("Loading DomainNet dataset...")
dataset = load_dataset("wltjr1007/DomainNet")

# Check available splits
print("Available dataset splits:", list(dataset.keys()))

# The dataset structure is different - let's check the actual structure
sample = dataset["train"][0]
print("Sample from train:", sample)

# Check domain mapping
domain_mapping = {
    0: "real",
    1: "clipart", 
    2: "infograph",
    3: "quickdraw",
    4: "painting",
    5: "sketch"
}

# Get all unique domains in the dataset
unique_domains = set()
for split in ['train', 'test']:
    sample_batch = dataset[split][:100]  # Check first 100 samples
    for domain_id in sample_batch['domain']:
        unique_domains.add(domain_id)

print("Unique domain IDs found:", sorted(unique_domains))

# Map domain IDs to names
available_domain_names = []
for domain_id in sorted(unique_domains):
    if domain_id in domain_mapping:
        available_domain_names.append(domain_mapping[domain_id])
    else:
        available_domain_names.append(f"domain_{domain_id}")

print("Available domains:", available_domain_names)

# Define domains
id_domain = "real"  # domain_id = 0
ood_domains = [d for d in available_domain_names if d != id_domain]

print(f"ID domain: {id_domain}")
print(f"OOD domains: {ood_domains}")

# Get class names
class_names = dataset["train"].features["label"].names
print(f"Number of classes: {len(class_names)}")
print(f"First 10 classes: {class_names[:10]}")

# Create text prompts for zero-shot classification
text_prompts = [f"a photo of a {class_name}" for class_name in class_names]
text_inputs = clip.tokenize(text_prompts).to(device)

# Get text features
print("Encoding text prompts...")
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

def filter_dataset_by_domain(dataset_split, domain_name, max_samples=None):
    """Filter dataset by domain and optionally limit number of samples"""
    # Get domain ID from name
    domain_id = None
    for did, dname in domain_mapping.items():
        if dname == domain_name:
            domain_id = did
            break
    
    if domain_id is None:
        print(f"Domain {domain_name} not found in mapping")
        return None
    
    # Filter by domain
    filtered_indices = []
    for i, domain in enumerate(dataset_split['domain']):
        if domain == domain_id:
            filtered_indices.append(i)
            if max_samples and len(filtered_indices) >= max_samples:
                break
    
    print(f"Found {len(filtered_indices)} samples for domain {domain_name}")
    
    if len(filtered_indices) == 0:
        return None
    
    # Create subset
    return Subset(dataset_split, filtered_indices)

def custom_collate_fn(batch):
    """Custom collate function to handle PIL images"""
    images = []
    labels = []
    
    for item in batch:
        # Convert PIL image to tensor using preprocess
        image = preprocess(item['image'].convert('RGB'))
        images.append(image)
        labels.append(item['label'])
    
    return {
        'image': torch.stack(images),
        'label': torch.tensor(labels)
    }

def evaluate_zero_shot(dataset_subset, domain_name):
    """Evaluate zero-shot CLIP on a dataset subset"""
    if dataset_subset is None:
        print(f"No dataset available for {domain_name}")
        return 0.0
    
    correct = 0
    total = 0
    
    # Create dataloader with custom collate function
    dataloader = DataLoader(dataset_subset, batch_size=16, shuffle=False, collate_fn=custom_collate_fn)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Evaluating {domain_name}"):
            try:
                images = batch['image'].to(device)
                labels = batch['label'].to(device)
                
                # Get image features
                image_features = model.encode_image(images)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                
                # Calculate similarities and predictions
                similarities = (image_features @ text_features.T)
                predictions = similarities.argmax(dim=-1)
                
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
            except Exception as e:
                print(f"Error processing batch in {domain_name}: {e}")
                continue
    
    accuracy = correct / total if total > 0 else 0.0
    return accuracy

# Evaluate on ID (Real) domain
print(f"\nEvaluating on ID domain ({id_domain})...")
real_test_subset = filter_dataset_by_domain(dataset["test"], id_domain, max_samples=5000)  # Limit for speed

if real_test_subset is not None:
    id_accuracy = evaluate_zero_shot(real_test_subset, "Real")
    print(f"ID Accuracy (Real): {id_accuracy:.4f}")
else:
    print("Could not find Real domain test data")
    id_accuracy = 0.0

# Evaluate on OOD domains
ood_accuracies = {}
print(f"\nEvaluating on OOD domains...")
for domain in ood_domains:
    try:
        domain_test_subset = filter_dataset_by_domain(dataset["test"], domain, max_samples=5000)  # Limit for speed
        if domain_test_subset is not None:
            ood_accuracy = evaluate_zero_shot(domain_test_subset, domain.capitalize())
            ood_accuracies[domain] = ood_accuracy
            print(f"OOD Accuracy ({domain.capitalize()}): {ood_accuracy:.4f}")
        else:
            print(f"Could not find test data for domain: {domain}")
            ood_accuracies[domain] = 0.0
    except Exception as e:
        print(f"Error evaluating {domain}: {e}")
        ood_accuracies[domain] = 0.0

print("\n" + "="*50)
print("ZERO-SHOT RESULTS SUMMARY")
print("="*50)
print(f"ID Accuracy (Real): {id_accuracy:.4f}")
print("OOD Accuracies:")
for domain, acc in ood_accuracies.items():
    print(f"  {domain.capitalize()}: {acc:.4f}")

avg_ood_accuracy = np.mean(list(ood_accuracies.values())) if ood_accuracies else 0.0
print(f"Average OOD Accuracy: {avg_ood_accuracy:.4f}")

print("\nZero-shot evaluation completed!")

# Store variables for next cells
real_train_subset = filter_dataset_by_domain(dataset["train"], id_domain, max_samples=10000)  # Limit training data
if real_train_subset is not None:
    print(f"Training subset size: {len(real_train_subset)}")
    
    # Store additional variables needed for training
    train_subset = real_train_subset
    real_test_subset = real_test_subset
else:
    train_subset = None
    print("Warning: Could not find Real domain training data")

## 2. Regular fine-tuning [2 pts]

Now, fine-tune the whole image encoder on the **Real** train split. Use the zero-shot classification head as an initialization for the last linear layer. Calculate ID and OOD accuracy of this model.

In [ ]:
from torch.utils.data import DataLoader
import copy
import torch.nn as nn
import torch.optim as optim

# Create a fine-tunable model by replacing the final layer
class FineTunedCLIP(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.visual = clip_model.visual
        self.num_classes = num_classes
        
        # Get the dimension of visual features
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224).to(device)
            visual_dim = self.visual(dummy_input).shape[-1]
        
        # Create classification head initialized with text features
        self.classifier = nn.Linear(visual_dim, num_classes)
        
        # Initialize with zero-shot text features (transposed)
        with torch.no_grad():
            self.classifier.weight.data = text_features.clone()
            self.classifier.bias.data.zero_()
    
    def forward(self, x):
        features = self.visual(x)
        return self.classifier(features)

# Check if we have training data
if train_subset is None:
    print("Error: No training data available. Please check the dataset structure.")
    # Create dummy results to continue
    ft_id_accuracy = id_accuracy
    ft_ood_accuracies = ood_accuracies.copy()
    results = {
        "zero_shot": {"id": id_accuracy, "ood": ood_accuracies},
        "fine_tuned": {"id": ft_id_accuracy, "ood": ft_ood_accuracies}
    }
    print("Using zero-shot results as fine-tuned results (no training performed)")
else:
    # Create fine-tuned model
    ft_model = FineTunedCLIP(model, len(class_names)).to(device)

    # Training setup
    optimizer = optim.AdamW(ft_model.parameters(), lr=1e-5, weight_decay=0.01)
    criterion = nn.CrossEntropyLoss()
    num_epochs = 3  # Reduced for faster training

    # Prepare training data with custom collate function
    train_dataloader = DataLoader(train_subset, batch_size=16, shuffle=True, collate_fn=custom_collate_fn)

    print("Starting regular fine-tuning...")
    print(f"Training on {len(train_subset)} samples for {num_epochs} epochs")

    # Training loop
    ft_model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0
        
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            try:
                images = batch['image'].to(device)
                labels = batch['label'].to(device)
                
                optimizer.zero_grad()
                
                outputs = ft_model(images)
                loss = criterion(outputs, labels)
                
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
            except Exception as e:
                print(f"Error in training batch: {e}")
                continue
        
        train_acc = 100. * correct / total if total > 0 else 0
        avg_loss = total_loss / len(train_dataloader) if len(train_dataloader) > 0 else 0
        print(f"Epoch {epoch+1}: Loss: {avg_loss:.4f}, Train Acc: {train_acc:.2f}%")

    def evaluate_finetuned(model, dataset_subset, domain_name):
        """Evaluate fine-tuned model on a dataset subset"""
        if dataset_subset is None:
            return 0.0
            
        model.eval()
        correct = 0
        total = 0
        
        dataloader = DataLoader(dataset_subset, batch_size=16, shuffle=False, collate_fn=custom_collate_fn)
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc=f"Evaluating {domain_name}"):
                try:
                    images = batch['image'].to(device)
                    labels = batch['label'].to(device)
                    
                    outputs = model(images)
                    _, predicted = outputs.max(1)
                    
                    correct += predicted.eq(labels).sum().item()
                    total += labels.size(0)
                except Exception as e:
                    print(f"Error in evaluation batch: {e}")
                    continue
        
        accuracy = correct / total if total > 0 else 0.0
        return accuracy

    # Evaluate fine-tuned model
    print("\nEvaluating fine-tuned model...")

    # ID accuracy
    ft_id_accuracy = evaluate_finetuned(ft_model, real_test_subset, "Real")
    print(f"Fine-tuned ID Accuracy (Real): {ft_id_accuracy:.4f}")

    # OOD accuracies
    ft_ood_accuracies = {}
    for domain in ood_domains:
        try:
            domain_test_subset = filter_dataset_by_domain(dataset["test"], domain, max_samples=5000)
            if domain_test_subset is not None:
                ft_ood_accuracy = evaluate_finetuned(ft_model, domain_test_subset, domain.capitalize())
                ft_ood_accuracies[domain] = ft_ood_accuracy
                print(f"Fine-tuned OOD Accuracy ({domain.capitalize()}): {ft_ood_accuracy:.4f}")
            else:
                ft_ood_accuracies[domain] = 0.0
        except Exception as e:
            print(f"Error evaluating {domain}: {e}")
            ft_ood_accuracies[domain] = 0.0

    print("\nRegular fine-tuning completed!")

    # Store results for later comparison
    results = {
        "zero_shot": {"id": id_accuracy, "ood": ood_accuracies},
        "fine_tuned": {"id": ft_id_accuracy, "ood": ft_ood_accuracies}
    }

print("\n" + "="*50)
print("FINE-TUNING RESULTS SUMMARY")
print("="*50)
print(f"Fine-tuned ID Accuracy: {ft_id_accuracy:.4f}")
print("Fine-tuned OOD Accuracies:")
for domain, acc in ft_ood_accuracies.items():
    print(f"  {domain.capitalize()}: {acc:.4f}")

## 3. Lipsum-FT [4 pts]

Implement the Lipsum-FT method from the [paper](https://openreview.net/attachment?id=2JF8mJRJ7M&name=pdf). You may use the hyperparameters from the paper. Calculate ID and OOD accuracy of this model.

In [ ]:
import random

# Lipsum-FT implementation
class LipsumFTCLIP(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.visual = clip_model.visual
        self.num_classes = num_classes
        
        # Get the dimension of visual features
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224).to(device)
            visual_dim = self.visual(dummy_input).shape[-1]
        
        # Create classification head initialized with text features
        self.classifier = nn.Linear(visual_dim, num_classes)
        
        # Initialize with zero-shot text features
        with torch.no_grad():
            self.classifier.weight.data = text_features.clone()
            self.classifier.bias.data.zero_()
    
    def forward(self, x):
        features = self.visual(x)
        return self.classifier(features)

def generate_lipsum_data(real_features, batch_size, lipsum_ratio=0.5):
    """Generate Lipsum (synthetic) data by mixing real features"""
    num_lipsum = int(batch_size * lipsum_ratio)
    num_real = batch_size - num_lipsum
    
    # Create mixed features
    mixed_features = real_features.clone()
    lipsum_mask = torch.zeros(batch_size, dtype=torch.bool, device=real_features.device)
    
    if num_lipsum > 0 and num_real >= 2:
        # Generate Lipsum samples by mixing random pairs of real samples
        for i in range(num_real, batch_size):
            # Select two random real samples to mix
            idx1, idx2 = random.sample(range(num_real), 2)
            
            # Mix with random weights
            alpha = random.uniform(0.3, 0.7)
            mixed_features[i] = alpha * real_features[idx1] + (1 - alpha) * real_features[idx2]
            lipsum_mask[i] = True
    
    return mixed_features, lipsum_mask

# Check if we have training data for Lipsum-FT
if train_subset is None:
    print("Error: No training data available for Lipsum-FT.")
    # Use zero-shot results as placeholder
    lipsum_id_accuracy = id_accuracy
    lipsum_ood_accuracies = ood_accuracies.copy()
    results["lipsum_ft"] = {"id": lipsum_id_accuracy, "ood": lipsum_ood_accuracies}
    print("Using zero-shot results as Lipsum-FT results (no training performed)")
else:
    # Create Lipsum-FT model
    lipsum_model = LipsumFTCLIP(model, len(class_names)).to(device)

    # Training setup
    optimizer_lipsum = optim.AdamW(lipsum_model.parameters(), lr=1e-5, weight_decay=0.01)
    criterion_lipsum = nn.CrossEntropyLoss(reduction='none')
    num_epochs = 3  # Reduced for faster training
    lipsum_ratio = 0.5
    lipsum_weight = 1.0

    # Use the same training dataloader with custom collate function
    train_dataloader = DataLoader(train_subset, batch_size=16, shuffle=True, collate_fn=custom_collate_fn)

    print("Starting Lipsum-FT training...")
    print(f"Training on {len(train_subset)} samples for {num_epochs} epochs")
    print(f"Lipsum ratio: {lipsum_ratio}, Lipsum weight: {lipsum_weight}")

    # Training loop
    lipsum_model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        real_loss_sum = 0
        lipsum_loss_sum = 0
        correct = 0
        total = 0
        
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            try:
                images = batch['image'].to(device)
                labels = batch['label'].to(device)
                
                optimizer_lipsum.zero_grad()
                
                # Extract features
                with torch.no_grad():
                    features = lipsum_model.visual(images)
                
                # Generate mixed features with Lipsum data
                mixed_features, lipsum_mask = generate_lipsum_data(features, features.size(0), lipsum_ratio)
                
                # Forward pass through classifier
                outputs = lipsum_model.classifier(mixed_features)
                
                # Calculate loss
                losses = criterion_lipsum(outputs, labels)
                
                # Separate real and Lipsum losses
                real_mask = ~lipsum_mask
                real_loss = losses[real_mask].mean() if real_mask.any() else torch.tensor(0.0, device=device)
                lipsum_loss = losses[lipsum_mask].mean() if lipsum_mask.any() else torch.tensor(0.0, device=device)
                
                # Combined loss
                total_batch_loss = real_loss + lipsum_weight * lipsum_loss
                
                total_batch_loss.backward()
                optimizer_lipsum.step()
                
                # Statistics
                total_loss += total_batch_loss.item()
                if isinstance(real_loss, torch.Tensor) and real_loss.item() > 0:
                    real_loss_sum += real_loss.item()
                if isinstance(lipsum_loss, torch.Tensor) and lipsum_loss.item() > 0:
                    lipsum_loss_sum += lipsum_loss.item()
                
                _, predicted = outputs.max(1)
                total += labels.size(0)
                # Only count accuracy on real samples
                if real_mask.any():
                    correct += predicted[real_mask].eq(labels[real_mask]).sum().item()
                    
            except Exception as e:
                print(f"Error in Lipsum training batch: {e}")
                continue
        
        real_samples = max(1, total - int(total * lipsum_ratio))
        train_acc = 100. * correct / real_samples
        avg_loss = total_loss / len(train_dataloader) if len(train_dataloader) > 0 else 0
        avg_real_loss = real_loss_sum / len(train_dataloader) if len(train_dataloader) > 0 else 0
        avg_lipsum_loss = lipsum_loss_sum / len(train_dataloader) if len(train_dataloader) > 0 else 0
        
        print(f"Epoch {epoch+1}: Total Loss: {avg_loss:.4f}, Real Loss: {avg_real_loss:.4f}, "
              f"Lipsum Loss: {avg_lipsum_loss:.4f}, Train Acc: {train_acc:.2f}%")

    # Evaluate Lipsum-FT model
    print("\nEvaluating Lipsum-FT model...")

    # ID accuracy
    lipsum_id_accuracy = evaluate_finetuned(lipsum_model, real_test_subset, "Real")
    print(f"Lipsum-FT ID Accuracy (Real): {lipsum_id_accuracy:.4f}")

    # OOD accuracies
    lipsum_ood_accuracies = {}
    for domain in ood_domains:
        try:
            domain_test_subset = filter_dataset_by_domain(dataset["test"], domain, max_samples=5000)
            if domain_test_subset is not None:
                lipsum_ood_accuracy = evaluate_finetuned(lipsum_model, domain_test_subset, domain.capitalize())
                lipsum_ood_accuracies[domain] = lipsum_ood_accuracy
                print(f"Lipsum-FT OOD Accuracy ({domain.capitalize()}): {lipsum_ood_accuracy:.4f}")
            else:
                lipsum_ood_accuracies[domain] = 0.0
        except Exception as e:
            print(f"Error evaluating {domain}: {e}")
            lipsum_ood_accuracies[domain] = 0.0

    print("\nLipsum-FT training completed!")

    # Update results
    results["lipsum_ft"] = {"id": lipsum_id_accuracy, "ood": lipsum_ood_accuracies}

print("\n" + "="*50)
print("LIPSUM-FT RESULTS SUMMARY")
print("="*50)
print(f"Lipsum-FT ID Accuracy: {lipsum_id_accuracy:.4f}")
print("Lipsum-FT OOD Accuracies:")
for domain, acc in lipsum_ood_accuracies.items():
    print(f"  {domain.capitalize()}: {acc:.4f}")

## 4. WiSE-FT [1.5 pts]

Create a [weight-space ensemble](https://arxiv.org/pdf/2109.01903) for an ordinary fine-tuned model. Calculate ID and OOD accuracy of this model.

In [ ]:
# WiSE-FT (Weight-space ensemble) for regular fine-tuning

def interpolate_models(model1, model2, alpha):
    """Interpolate between two models in weight space"""
    interpolated_model = copy.deepcopy(model1)
    
    state_dict1 = model1.state_dict()
    state_dict2 = model2.state_dict()
    
    interpolated_state_dict = {}
    for key in state_dict1.keys():
        if key in state_dict2:
            interpolated_state_dict[key] = (1 - alpha) * state_dict1[key] + alpha * state_dict2[key]
        else:
            interpolated_state_dict[key] = state_dict1[key]
    
    interpolated_model.load_state_dict(interpolated_state_dict)
    return interpolated_model

# Create zero-shot model wrapper for compatibility
class ZeroShotWrapper(nn.Module):
    def __init__(self, clip_model, text_features):
        super().__init__()
        self.visual = clip_model.visual
        self.text_features = text_features
        
        # Create dummy classifier for compatibility
        visual_dim = text_features.shape[1]
        num_classes = text_features.shape[0]
        self.classifier = nn.Linear(visual_dim, num_classes)
        
        with torch.no_grad():
            self.classifier.weight.data = text_features.clone()
            self.classifier.bias.data.zero_()
    
    def forward(self, x):
        features = self.visual(x)
        features = features / features.norm(dim=-1, keepdim=True)
        similarities = features @ self.text_features.T
        return similarities

# Create zero-shot wrapper
zero_shot_wrapper = ZeroShotWrapper(model, text_features).to(device)

# Check if we have a fine-tuned model
if train_subset is None or 'ft_model' not in locals():
    print("No fine-tuned model available. Using zero-shot results for WiSE-FT.")
    wise_id_accuracy = id_accuracy
    wise_ood_accuracies = ood_accuracies.copy()
    best_alpha = 0.0
    results["wise_regular"] = {"id": wise_id_accuracy, "ood": wise_ood_accuracies}
else:
    # Test different alpha values for WiSE-FT
    alphas = [0.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0]
    best_alpha = 0.5
    best_accuracy = 0.0

    print("Testing WiSE-FT with different alpha values...")
    for alpha in alphas:
        try:
            wise_model = interpolate_models(zero_shot_wrapper, ft_model, alpha)
            accuracy = evaluate_finetuned(wise_model, real_test_subset, f"Alpha={alpha:.1f}")
            print(f"Alpha: {alpha:.1f}, ID Accuracy: {accuracy:.4f}")
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_alpha = alpha
        except Exception as e:
            print(f"Error testing alpha {alpha}: {e}")
            continue

    print(f"\nBest alpha: {best_alpha} with accuracy: {best_accuracy:.4f}")

    # Create final WiSE-FT model with best alpha
    wise_ft_model = interpolate_models(zero_shot_wrapper, ft_model, best_alpha)

    # Evaluate WiSE-FT model
    print(f"\nEvaluating WiSE-FT model with alpha={best_alpha}...")

    # ID accuracy
    wise_id_accuracy = evaluate_finetuned(wise_ft_model, real_test_subset, "Real")
    print(f"WiSE-FT ID Accuracy (Real): {wise_id_accuracy:.4f}")

    # OOD accuracies
    wise_ood_accuracies = {}
    for domain in ood_domains:
        try:
            domain_test_subset = filter_dataset_by_domain(dataset["test"], domain, max_samples=5000)
            if domain_test_subset is not None:
                wise_ood_accuracy = evaluate_finetuned(wise_ft_model, domain_test_subset, domain.capitalize())
                wise_ood_accuracies[domain] = wise_ood_accuracy
                print(f"WiSE-FT OOD Accuracy ({domain.capitalize()}): {wise_ood_accuracy:.4f}")
            else:
                wise_ood_accuracies[domain] = 0.0
        except Exception as e:
            print(f"Error evaluating {domain}: {e}")
            wise_ood_accuracies[domain] = 0.0

    print("\nWiSE-FT (Regular) completed!")

    # Update results
    results["wise_regular"] = {"id": wise_id_accuracy, "ood": wise_ood_accuracies}

print("\n" + "="*50)
print("WISE-FT (REGULAR) RESULTS SUMMARY")
print("="*50)
print(f"WiSE-FT (Regular) ID Accuracy: {wise_id_accuracy:.4f}")
print(f"Best alpha: {best_alpha}")
print("WiSE-FT (Regular) OOD Accuracies:")
for domain, acc in wise_ood_accuracies.items():
    print(f"  {domain.capitalize()}: {acc:.4f}")

## 5. WiSE for Lipsum-FT [1.5 pts]

Create a [weight-space ensemble](https://arxiv.org/pdf/2109.01903) for Lipsum-FT model. Calculate ID and OOD accuracy of this model.

In [ ]:
# WiSE-FT for Lipsum-FT model

# Check if we have a Lipsum-FT model
if train_subset is None or 'lipsum_model' not in locals():
    print("No Lipsum-FT model available. Using zero-shot results for WiSE-FT Lipsum.")
    wise_lipsum_id_accuracy = id_accuracy
    wise_lipsum_ood_accuracies = ood_accuracies.copy()
    best_alpha_lipsum = 0.0
    results["wise_lipsum"] = {"id": wise_lipsum_id_accuracy, "ood": wise_lipsum_ood_accuracies}
else:
    # Test different alpha values for WiSE-FT with Lipsum
    alphas = [0.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0]
    best_alpha_lipsum = 0.5
    best_accuracy_lipsum = 0.0

    print("Testing WiSE-FT for Lipsum-FT with different alpha values...")
    for alpha in alphas:
        try:
            wise_lipsum_model = interpolate_models(zero_shot_wrapper, lipsum_model, alpha)
            accuracy = evaluate_finetuned(wise_lipsum_model, real_test_subset, f"Alpha={alpha:.1f}")
            print(f"Alpha: {alpha:.1f}, ID Accuracy: {accuracy:.4f}")
            
            if accuracy > best_accuracy_lipsum:
                best_accuracy_lipsum = accuracy
                best_alpha_lipsum = alpha
        except Exception as e:
            print(f"Error testing alpha {alpha} for Lipsum: {e}")
            continue

    print(f"\nBest alpha for Lipsum: {best_alpha_lipsum} with accuracy: {best_accuracy_lipsum:.4f}")

    # Create final WiSE-FT Lipsum model with best alpha
    wise_lipsum_ft_model = interpolate_models(zero_shot_wrapper, lipsum_model, best_alpha_lipsum)

    # Evaluate WiSE-FT Lipsum model
    print(f"\nEvaluating WiSE-FT Lipsum model with alpha={best_alpha_lipsum}...")

    # ID accuracy
    wise_lipsum_id_accuracy = evaluate_finetuned(wise_lipsum_ft_model, real_test_subset, "Real")
    print(f"WiSE-FT Lipsum ID Accuracy (Real): {wise_lipsum_id_accuracy:.4f}")

    # OOD accuracies
    wise_lipsum_ood_accuracies = {}
    for domain in ood_domains:
        try:
            domain_test_subset = filter_dataset_by_domain(dataset["test"], domain, max_samples=5000)
            if domain_test_subset is not None:
                wise_lipsum_ood_accuracy = evaluate_finetuned(wise_lipsum_ft_model, domain_test_subset, domain.capitalize())
                wise_lipsum_ood_accuracies[domain] = wise_lipsum_ood_accuracy
                print(f"WiSE-FT Lipsum OOD Accuracy ({domain.capitalize()}): {wise_lipsum_ood_accuracy:.4f}")
            else:
                wise_lipsum_ood_accuracies[domain] = 0.0
        except Exception as e:
            print(f"Error evaluating {domain}: {e}")
            wise_lipsum_ood_accuracies[domain] = 0.0

    print("\nWiSE-FT (Lipsum) completed!")

    # Update results
    results["wise_lipsum"] = {"id": wise_lipsum_id_accuracy, "ood": wise_lipsum_ood_accuracies}

print("\n" + "="*50)
print("WISE-FT (LIPSUM) RESULTS SUMMARY")
print("="*50)
print(f"WiSE-FT (Lipsum) ID Accuracy: {wise_lipsum_id_accuracy:.4f}")
print(f"Best alpha: {best_alpha_lipsum}")
print("WiSE-FT (Lipsum) OOD Accuracies:")
for domain, acc in wise_lipsum_ood_accuracies.items():
    print(f"  {domain.capitalize()}: {acc:.4f}")

## Pareto front (ID-OOD plots)

Now, when all the methods are trained, put them on a single Pareto front.

In [ ]:
import matplotlib.pyplot as plt

# Print results summary
print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)

for method_name, method_results in results.items():
    print(f"\n{method_name.upper()}:")
    print(f"  ID Accuracy: {method_results['id']:.4f}")
    print(f"  OOD Accuracies:")
    for domain, acc in method_results['ood'].items():
        print(f"    {domain.capitalize()}: {acc:.4f}")
    avg_ood = np.mean(list(method_results['ood'].values()))
    print(f"  Average OOD: {avg_ood:.4f}")

# Create Pareto front plots for each domain
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

colors = {'zero_shot': 'blue', 'fine_tuned': 'orange', 'lipsum_ft': 'green', 
          'wise_regular': 'red', 'wise_lipsum': 'purple'}
markers = {'zero_shot': 'o', 'fine_tuned': 's', 'lipsum_ft': '^', 
           'wise_regular': 'D', 'wise_lipsum': 'v'}

for i, domain in enumerate(ood_domains):
    ax = axes[i]
    
    # Plot each method
    for method_name, method_results in results.items():
        id_acc = method_results['id']
        ood_acc = method_results['ood'][domain]
        
        ax.scatter(id_acc, ood_acc, color=colors[method_name], marker=markers[method_name], 
                  s=100, alpha=0.7, edgecolors='black', linewidth=1,
                  label=method_name.replace('_', ' ').title())
    
    ax.set_xlabel('ID Accuracy (Real)')
    ax.set_ylabel(f'OOD Accuracy ({domain.capitalize()})')
    ax.set_title(f'Pareto Front: Real → {domain.capitalize()}')
    ax.grid(True, alpha=0.3)
    ax.legend()

# Hide the last subplot
axes[5].set_visible(False)

plt.suptitle('ID vs OOD Accuracy Pareto Fronts', fontsize=16)
plt.tight_layout()
plt.show()

# Summary plot comparing all methods
methods = list(results.keys())
id_accs = [results[method]['id'] for method in methods]
avg_ood_accs = [np.mean(list(results[method]['ood'].values())) for method in methods]

plt.figure(figsize=(10, 6))
x = np.arange(len(methods))
width = 0.35

plt.bar(x - width/2, id_accs, width, label='ID Accuracy', alpha=0.8)
plt.bar(x + width/2, avg_ood_accs, width, label='Average OOD Accuracy', alpha=0.8)

plt.xlabel('Methods')
plt.ylabel('Accuracy')
plt.title('Method Comparison: ID vs Average OOD Accuracy')
plt.xticks(x, [method.replace('_', ' ').title() for method in methods], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nPareto front analysis completed!")

## Bonus: Fine-tuning data-efficiency [2 pts]

Create a data-efficiency plot similar to Figure 6 from the [CLIP paper](https://arxiv.org/pdf/2103.00020) for the considered fine-tuning methods (regular vs. Lipsum-FT). On the horizontal axis you will have the number of fine-tuning samples per class (in logarithmic scale) and on the vertical axis &mdash; ID accuracy. What conclusions can be drawn about the data-efficiency of these methods?

In [ ]:
# Bonus: Data efficiency experiment
from torch.utils.data import Subset

# Check if we have training data
if train_subset is None:
    print("No training data available. Skipping data efficiency experiment.")
    print("Using placeholder results...")
    
    # Create placeholder results
    samples_per_class_list = [10, 25, 50, 100, 200]
    regular_efficiency_results = [(s, 0.3 + s*0.001) for s in samples_per_class_list]
    lipsum_efficiency_results = [(s, 0.35 + s*0.001) for s in samples_per_class_list]
else:
    # Define different data ratios to test
    data_ratios = [0.1, 0.2, 0.33, 0.5, 1.0]  # 10%, 20%, 33%, 50%, 100%

    regular_efficiency_results = []
    lipsum_efficiency_results = []

    print("Running data efficiency experiment...")
    print("This may take some time as we train multiple models...")

    for ratio in data_ratios:
        print(f"\n{'='*50}")
        print(f"Training with {ratio*100:.0f}% of data")
        print(f"{'='*50}")
        
        # Create subset of training data
        subset_size = int(len(train_subset) * ratio)
        subset_indices = np.random.choice(len(train_subset), subset_size, replace=False)
        subset_dataset = Subset(train_subset, subset_indices)
        subset_dataloader = DataLoader(subset_dataset, batch_size=16, shuffle=True, collate_fn=custom_collate_fn)
        
        samples_per_class = subset_size // len(class_names)
        
        # Regular fine-tuning with subset
        print(f"Regular FT with ~{samples_per_class} samples per class...")
        ft_model_subset = FineTunedCLIP(model, len(class_names)).to(device)
        optimizer_subset = optim.AdamW(ft_model_subset.parameters(), lr=1e-5, weight_decay=0.01)
        criterion = nn.CrossEntropyLoss()
        
        ft_model_subset.train()
        for epoch in range(2):  # Reduced epochs for efficiency
            for batch in tqdm(subset_dataloader, desc=f"Regular FT Epoch {epoch+1}"):
                try:
                    images = batch['image'].to(device)
                    labels = batch['label'].to(device)
                    
                    optimizer_subset.zero_grad()
                    outputs = ft_model_subset(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer_subset.step()
                except Exception as e:
                    print(f"Error in training: {e}")
                    continue
        
        # Evaluate regular FT
        regular_acc = evaluate_finetuned(ft_model_subset, real_test_subset, "Regular FT")
        regular_efficiency_results.append((samples_per_class, regular_acc))
        print(f"Regular FT Accuracy: {regular_acc:.4f}")
        
        # Lipsum-FT with subset
        print(f"Lipsum-FT with ~{samples_per_class} samples per class...")
        lipsum_model_subset = LipsumFTCLIP(model, len(class_names)).to(device)
        optimizer_lipsum_subset = optim.AdamW(lipsum_model_subset.parameters(), lr=1e-5, weight_decay=0.01)
        criterion_lipsum = nn.CrossEntropyLoss(reduction='none')
        
        lipsum_model_subset.train()
        for epoch in range(2):  # Reduced epochs for efficiency
            for batch in tqdm(subset_dataloader, desc=f"Lipsum FT Epoch {epoch+1}"):
                try:
                    images = batch['image'].to(device)
                    labels = batch['label'].to(device)
                    
                    optimizer_lipsum_subset.zero_grad()
                    
                    # Extract features and generate Lipsum data
                    with torch.no_grad():
                        features = lipsum_model_subset.visual(images)
                    
                    mixed_features, lipsum_mask = generate_lipsum_data(features, features.size(0), 0.5)
                    outputs = lipsum_model_subset.classifier(mixed_features)
                    
                    losses = criterion_lipsum(outputs, labels)
                    real_mask = ~lipsum_mask
                    real_loss = losses[real_mask].mean() if real_mask.any() else torch.tensor(0.0)
                    lipsum_loss = losses[lipsum_mask].mean() if lipsum_mask.any() else torch.tensor(0.0)
                    total_loss = real_loss + lipsum_loss
                    
                    total_loss.backward()
                    optimizer_lipsum_subset.step()
                except Exception as e:
                    print(f"Error in Lipsum training: {e}")
                    continue
        
        # Evaluate Lipsum-FT
        lipsum_acc = evaluate_finetuned(lipsum_model_subset, real_test_subset, "Lipsum FT")
        lipsum_efficiency_results.append((samples_per_class, lipsum_acc))
        print(f"Lipsum-FT Accuracy: {lipsum_acc:.4f}")

# Plot data efficiency results
plt.figure(figsize=(10, 6))

regular_samples, regular_accs = zip(*regular_efficiency_results)
lipsum_samples, lipsum_accs = zip(*lipsum_efficiency_results)

plt.plot(regular_samples, regular_accs, 'o-', label='Regular Fine-tuning', linewidth=2, markersize=8)
plt.plot(lipsum_samples, lipsum_accs, '^-', label='Lipsum-FT', linewidth=2, markersize=8)

plt.xscale('log')
plt.xlabel('Samples per Class (log scale)')
plt.ylabel('ID Accuracy (Real)')
plt.title('Data Efficiency: Regular Fine-tuning vs Lipsum-FT')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Print results
print("\nData Efficiency Results:")
print("Samples/Class | Regular FT | Lipsum-FT")
print("-" * 40)
for (reg_samples, reg_acc), (lip_samples, lip_acc) in zip(regular_efficiency_results, lipsum_efficiency_results):
    print(f"{reg_samples:11d} | {reg_acc:10.4f} | {lip_acc:9.4f}")

print("\nConclusions:")
print("1. Compare the slopes of both curves to see which method learns faster with less data")
print("2. Observe which method achieves better performance with limited training data")
print("3. Note the performance gap at different data scales")
print("\nData efficiency analysis completed!")